##### Script for scraping charity data from Wikidata Public API
##### Author: Matthew Albert

In [ ]:
# Imports for workflow
import requests
import copy
import time
import json
from wikidata_utilities import *
from reliefweb_utilities import get_relevant_countries

In [ ]:
# Define the mapping from label to property ID in Wikidata pages
label_to_prop = {
  "org_type": "P31",
  "logo_img": "P154",
  "org_img": "P18",
  "established": "P571",
  "abbreviation": "P1813",
  "parent_org": "P749",
  "headquarters": "P159",
  "awards_received": "P166",
  "website": "P856"
}

# Define json structure that scraped data will be stored in 
charity_instance = {
  "name" : "",
  "id" : "",
  "attributes" : {
    "description" : "",
    "org_type": [],
    "logo_img": "",
    "org_img": "",
    "established": "",
    "short_name": "",
    "long_name": "",
    "parent_org": "",
    "headquarters": "",
    "hq_country": "",
    "awards_received": [],
    "website": "",
    "relief_web_id": "",
    "relief_web_url": "",
    "relief_provided": [],
    "relevant_countries": {
      "primary_countries": [],
      "secondary_countries": []
    },
    "youtube_info": {
      "channel_id": "",
      "channel_url": "",
      "relevant_videos": []
    }
  }
}

# Define outfile path
outfile = "models_data/charity_db.json"

WIKIDATA_URL = "https://www.wikidata.org/w/api.php"
RELIF_WEB_URL = "https://api.reliefweb.int/v1/sources"

##### Utility Functions

In [ ]:
# Function used to scrape wikidata and populate relevant fields
def populate_wikidata_fields(json_response, page_id, data):
  # Populate fields with data (or empty string if property does not exist for page)
  data["id"] = page_id
  if data["attributes"]["description"] == "":
    # Populate field with description from Wikidata page
    data["attributes"]["description"] = json_response["entities"][page_id]["descriptions"]["en"]["value"]
    
  # Use try/except to handle cases where property does not exist for page
  
  if (data["attributes"]["org_type"] == []):
    # Try to scrape organization type
    try:
      org_type_list = json_response["entities"][page_id]["claims"][label_to_prop["org_type"]]
      org_types = []
      for org in org_type_list:
        org_type = get_page_title(org["mainsnak"]["datavalue"]["value"]["id"])
        data["attributes"]["org_type"].append(org_type)
    except:
      pass
  
  # Try to scrape logo image
  try:
    img_file = json_response["entities"][page_id]["claims"][label_to_prop["logo_img"]][0]["mainsnak"]["datavalue"]["value"]
    data["attributes"]["logo_img"] = get_img_url(img_file)
  except:
    pass
  
  # Try to scrape for image related to organization
  try:
    img_file = json_response["entities"][page_id]["claims"][label_to_prop["org_img"]][0]["mainsnak"]["datavalue"]["value"]
    data["attributes"]["org_img"] = get_img_url(img_file)
  except:
    pass
  
  # Try to scrape the year the organization was established
  try:
    date = json_response["entities"][page_id]["claims"][label_to_prop["established"]][0]["mainsnak"]["datavalue"]["value"]["time"]
    data["attributes"]["established"] = date[1:]
  except:
    pass
  
  if data["attributes"]["short_name"] == "":
    # Try to scrape the abbreviation of the organization
    try:
      data["attributes"]["abbreviation"] = json_response["entities"][page_id]["claims"][label_to_prop["abbreviation"]][0]["mainsnak"]["datavalue"]["value"]["text"]
    except:
      pass
  
  # Try to scrape for the parent organization
  try:
    parent_org = json_response["entities"][page_id]["claims"][label_to_prop["parent_org"]][0]["mainsnak"]["datavalue"]["value"]["id"]
    data["attributes"]["parent_org"] = get_page_title(parent_org)
  except:
    pass
  
  # Try to scrape for headquarters of organization
  try:
    headquarters = json_response["entities"][page_id]["claims"][label_to_prop["headquarters"]][0]["mainsnak"]["datavalue"]["value"]["id"]
    data["attributes"]["headquarters"] = get_page_title(headquarters)
  except:
    pass
  
  # Try to scrape for awards received by organization
  try:
    awards_list = json_response["entities"][page_id]["claims"][label_to_prop["awards_received"]]
    for award in awards_list:
      award_dict = {
        "award_name": "",
        "award_date": ""
      }
      award_name = get_page_title(award["mainsnak"]["datavalue"]["value"]["id"])
      date_property = "P585"
      award_date = award["qualifiers"][date_property][0]["datavalue"]["value"]["time"][1:]
      # Populate fields of award json object
      award_dict["award_name"] = award_name
      award_dict["award_date"] = award_date
      data["attributes"]["awards_received"].append(award_dict)
  except:
    pass
  
  if data["attributes"]["website"] == "":
    # Try to scrap for website of organization
    try:
      data["attributes"]["website"] = json_response["entities"][page_id]["claims"][label_to_prop["website"]][0]["mainsnak"]["datavalue"]["value"]
    except:
      pass

# Function used to scrape data from ReliefWeb API
def populate_relief_web_fields(charity, data, alternate_params=None):
  # Define the parameters for the API request
  params = {
    "appname": "syrianrefugeecrisis",
    "profile": "full",
    "limit": 1,
    "query[value]": charity
  }
  if alternate_params != None:
    params = alternate_params
  # Make API request to get organization data
  response = requests.get(RELIF_WEB_URL, params=params)
  # Very response was a success
  if response.status_code == 200 and response.json()["count"] > 0:
    # Extract org data object
    org_data = response.json()["data"][0]
    # Populate fields with relevant data from API response
    
    # Use try/except to handle cases where field does not exist for organization
    try:
      data["attributes"]["relief_web_id"] = org_data["id"]
    except:
      pass
    
    try:
      data["attributes"]["description"] = org_data["fields"]["description"]
    except:
      pass
    
    try:
      data["attributes"]["org_type"].append(org_data["fields"]["type"]["name"])
    except:
      pass
    
    try:
      data["attributes"]["short_name"] = org_data["fields"]["shortname"]
    except:
      pass
    
    try:
      data["attributes"]["long_name"] = org_data["fields"]["longname"]
    except:
      pass
    
    try:
      data["attributes"]["website"] = org_data["fields"]["homepage"]
    except:
      pass
    
    try:
      data["attributes"]["hq_country"] = org_data["fields"]["country"][0]["name"]
    except:
      pass
    
    try:
      data["attributes"]["logo_img"] = org_data["fields"]["logo"]["url"]
    except:
      pass
    
    try:
      data["attributes"]["relief_web_url"] = org_data["fields"]["url_alias"]
    except:
      print(f"Failed to get ReliefWeb URL for {charity}")
      pass
    
  else:
    print(f"ReliefWeb API Request for {charity} failed")
  

In [ ]:
# Function used to scrape data for a single charity/organization
def scrape_charity(charity):
  # Extract fields from charity dict
  charity_name = charity["name"]
  relief_list = charity["relief"]
  
  # Create deep copy of charity_instance and use to populate with data
  charity_data = copy.deepcopy(charity_instance)
  
  # Populate fields that don't need to be scraped
  charity_data["name"] = charity_name
  charity_data["attributes"]["relief_provided"] = relief_list
  
  # First populate data from ReliefWeb
  if charity.get("alternate_query") is not None:
    # If alternate query is present, use that to query ReliefWeb API
    populate_relief_web_fields(charity["alternate_query"], charity_data)
  elif charity.get("query_by_id") is not None:
    # Charity requires using ReliefWeb ID to query API for charity data 
    params = {
      "appname": "syrianrefugeecrisis",
      "profile": "full",
      "limit": 1,
      "query[fields][]": "id",
      "query[value]": charity["query_by_id"]
    }
    populate_relief_web_fields(charity_name, charity_data, params)
  else:
    populate_relief_web_fields(charity_name, charity_data)
  
  # Populate data from Wikidata
  page_id = get_page_id(charity_name)
  page_data = get_page_data(page_id)
  populate_wikidata_fields(page_data, page_id, charity_data)
  
  # Populate relevant countries for charity
  charity_data["attributes"]["relevant_countries"] = get_relevant_countries(charity_name, charity_data["attributes"]["relief_web_id"])
  
  return charity_data

In [ ]:
# Function to scrape data for all charities
def scrape_all_charities():
  # Create list to store charity data
  charity_data = []
  charities = []
  # Load list of charities to scrape from json file
  with open("charities.json", "r") as f:
    charities = json.load(f)
  # Iterate through charities and scrape data
  count = 1
  for charity in charities:
    charity_name = charity["name"]
    print(f"Scraping data for {charity_name}...{count}/{len(charities)}")
    charity_data.append(scrape_charity(charity))
    count += 1
    time.sleep(5)
  return charity_data

# scraped_data = scrape_all_charities()

In [ ]:
# Function to write scraped data to JSON file
def write_to_json(data):
  # Write scraped data to JSON file
  with open(outfile, "w") as json_file:
    json.dump(data, json_file, indent=2)

# write_to_json(scraped_data)
# print(len(scraped_data))